In [52]:
from PIL import Image as Pimage
import cv2 as cv
import torch.nn as nn
import torch.utils.data as data
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
import torchvision.transforms as ToTensor
import matplotlib.pyplot as plt
import os
import pandas as pd
from torchvision.io import read_image

In [53]:
class CustomDataLoader(Dataset):

    def __init__(self,img_csv, img_dir, transform=None, target_transform=None):

        self.img_dir = img_dir
        self.img_labels = pd.read_csv(img_csv)
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = Pimage.open(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
    
    def __len__(self):
        return len(self.img_labels)

def from_image_to_label(image_dir):
    with open("dataset.csv", "w") as f:
        for img_name in os.listdir(image_dir):
            img_label = img_name.split(("-"))[0]
            f.write(img_name +"," + img_label + "\n")
    f.close()
    return ("dataset.csv")
        



In [54]:
img_dir = "../data/"
csv = from_image_to_label(img_dir)
t=ToTensor.Compose([ToTensor.ToTensor(), ToTensor.Resize([1200, 1200])])
image_dataset = CustomDataLoader(csv, img_dir,transform=t )
train_len = int(image_dataset.__len__()*0.8)
test_len = image_dataset.__len__() - train_len
train_set, test_set = data.random_split(image_dataset, [train_len, test_len])

train_dataloader = DataLoader(train_set, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=4, shuffle=True)


Feature batch shape: torch.Size([4, 3, 1200, 1200])
Labels batch shape: ('OK', 'OK', 'OK', 'NOK')


Label: OK


In [55]:
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels}")
img = train_features[0]
label = train_labels[0]
plt.imshow(img.permute(1, 2, 0),cmap='gray',vmin=0,vmax=255)
plt.show()
print(f"Label: {label}")

In [56]:
class customNet(nn.Module):
    def __init__(self):
        super(self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(3, 20, 5),
                                   nn.ReLU(),
                                   nn.Conv2d(20,64,5),
                                   nn.ReLU(),
                                   nn.Flatten())
        self.fullc = nn.Sequential(nn.Linear(4096,128),
                                   nn.Linear(128, 3))    

    def forward(self, x):
        x = self.conv1(x)
        x= self.fullc(x)
    


In [ ]:
device = "cuda"
model = customNet().to(device)
